读取txt

In [25]:
import pandas as pd
data_path='Gowalla_totalCheckins.txt'
columns = ['user_id', 'checkin_time', 'latitude', 'longitude', 'location_id']
df=pd.read_csv(data_path,sep='\s+',names=columns)
print(df.head())

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
D:\Users\Wind\AppData\Local\Temp\ipykernel_19448\2458330053.py:4: SyntaxWarning: invalid escape sequence '\s'
  df=pd.read_csv(data_path,sep='\s+',names=columns)


   user_id          checkin_time   latitude  longitude  location_id
0        0  2010-10-19T23:55:27Z  30.235909 -97.795140        22847
1        0  2010-10-18T22:17:43Z  30.269103 -97.749395       420315
2        0  2010-10-17T23:42:03Z  30.255731 -97.763386       316637
3        0  2010-10-17T19:26:05Z  30.263418 -97.757597        16516
4        0  2010-10-16T18:50:42Z  30.274292 -97.740523      5535878


In [26]:
import numpy as np
RANDOM_SEED = 0
NEG_ITEMS = 99

In [27]:
df['checkin_time']=pd.to_datetime(df['checkin_time'])
df['time_numeric']=df['checkin_time'].dt.strftime('%Y%m%d%H%M%S')
print(df.head())

   user_id              checkin_time   latitude  longitude  location_id  \
0        0 2010-10-19 23:55:27+00:00  30.235909 -97.795140        22847   
1        0 2010-10-18 22:17:43+00:00  30.269103 -97.749395       420315   
2        0 2010-10-17 23:42:03+00:00  30.255731 -97.763386       316637   
3        0 2010-10-17 19:26:05+00:00  30.263418 -97.757597        16516   
4        0 2010-10-16 18:50:42+00:00  30.274292 -97.740523      5535878   

     time_numeric  
0  20101019235527  
1  20101018221743  
2  20101017234203  
3  20101017192605  
4  20101016185042  


In [28]:
n_users = df['user_id'].value_counts().size
n_items = df['location_id'].value_counts().size
n_clicks = len(df)
min_time = df['time_numeric'].min()
max_time = df['time_numeric'].max()
print(n_users, n_items, n_clicks, min_time, max_time)

107092 1280969 6442892 20090204051738 20101023052206


In [31]:
np.random.seed(RANDOM_SEED)

In [ ]:
out_df = df.rename(columns={'location_id': 'item_id', 'user_id': 'user_id', 'time_numeric': 'time'})
out_df = out_df[['user_id', 'item_id', 'time']]
out_df = out_df.drop_duplicates(['user_id', 'item_id', 'time'])
out_df = out_df.sort_values(by=['time', 'user_id'], kind='mergesort').reset_index(drop=True)
l_u=int(len(out_df['user_id'].unique())/6)
l_i=int(len(out_df['item_id'].unique())/6)

np.random.seed(42)
sample_users=np.random.choice(out_df['user_id'].unique(),l_u,replace=False)
sample_items=np.random.choice(out_df['item_id'].unique(),l_i,replace=False)

filtered_df=out_df[out_df['user_id'].isin(sample_users) & out_df['item_id'].isin(sample_items)]

out_df=filtered_df
out_df.head()

,user_id,item_id,time
154,290,9224,20090315062646
156,197,9224,20090315063406
157,290,9299,20090315063519
470,290,9221,20090317042616
471,197,9221,20090317042647


In [33]:
uids = sorted(out_df['user_id'].unique())
user2id = dict(zip(uids, range(1, len(uids) + 1)))
iids = sorted(out_df['item_id'].unique())
item2id = dict(zip(iids, range(1, len(iids) + 1)))

out_df['user_id'] = out_df['user_id'].apply(lambda x: user2id[x])
out_df['item_id'] = out_df['item_id'].apply(lambda x: item2id[x])
out_df.head()

,user_id,item_id,time
154,56,30,20090315062646
156,35,30,20090315063406
157,56,36,20090315063519
470,56,29,20090317042616
471,35,29,20090317042647


In [34]:
# leave one out spliting

clicked_item_set = dict()
for user_id, seq_df in out_df.groupby('user_id'):
    clicked_item_set[user_id] = set(seq_df['item_id'].values.tolist())
    
def generate_dev_test(data_df):
    result_dfs = []
    n_items = data_df['item_id'].value_counts().size
    for idx in range(2):
        result_df = data_df.groupby('user_id').tail(1).copy()
        data_df = data_df.drop(result_df.index)
        neg_items = np.random.randint(1, n_items + 1, (len(result_df), NEG_ITEMS))
        for i, uid in enumerate(result_df['user_id'].values):
            user_clicked = clicked_item_set[uid]
            for j in range(len(neg_items[i])):
                while neg_items[i][j] in user_clicked:
                    neg_items[i][j] = np.random.randint(1, n_items + 1)
        result_df['neg_items'] = neg_items.tolist()
        result_dfs.append(result_df)
    return result_dfs, data_df

In [35]:
import numpy as np
leave_df = out_df.groupby('user_id').head(1)
data_df = out_df.drop(leave_df.index)

[test_df, dev_df], data_df = generate_dev_test(data_df)
train_df = pd.concat([leave_df, data_df]).sort_index()

len(train_df), len(dev_df), len(test_df)

(224293, 12853, 14705)

In [36]:
train_df.head()

,user_id,item_id,time
154,56,30,20090315062646
156,35,30,20090315063406
157,56,36,20090315063519
470,56,29,20090317042616
471,35,29,20090317042647


In [37]:
test_df.head()

,user_id,item_id,time,neg_items
2423,4905,656,20090530193818,"[32833, 58563, 59228, 10760, 27061, 8513, 5025..."
2792,12810,776,20090614135850,"[36559, 63190, 52048, 41863, 74456, 35065, 296..."
4107,4381,1167,20090729170051,"[48356, 12963, 45855, 4280, 5390, 58967, 84236..."
4373,4324,545,20090803185413,"[34541, 30116, 6447, 68615, 78624, 87859, 6721..."
11563,7996,2218,20090927214716,"[72038, 4129, 42308, 29740, 65234, 46637, 4564..."


In [38]:
# save results

train_df.to_csv('train.csv', sep='\t', index=False)
dev_df.to_csv('dev.csv', sep='\t', index=False)
test_df.to_csv('test.csv', sep='\t', index=False)